# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company,
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company,
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
http

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'portfolio project',
   'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'portfolio project', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook profile',
   'url': 'https://www.facebook.com/edward.donner.52'},
  {'type': 'patent',
   'url': 'https://patents.google.com/patent/US20210049536A1/'}]}

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [ ]:
select_relevant_links("https://edwarddonner.com")

In [10]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links


{'links': [{'type': 'brand/about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'join page', 'url': 'https://huggingface.co/join'},
  {'type': 'Discord community', 'url': 'https://huggingface.co/join/discord'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'Discussion forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'Status page', 'url': 'https://status.huggingface.co/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [11]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [13]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 4 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Tongyi-MAI/Z-Image-Turbo
Updated
5 days ago
•
269k
•
2.64k
microsoft/VibeVoice-Realtime-0.5B
Updated
1 day ago
•
120k
•
801
zai-org/GLM-4.6V-Flash
Updated
4 days ago
•
54.1k
•
390
mistralai/Devstral-Small-2-24B-Instruct-2512
Updated
1 day ago
•
13.7k
•
313
zai-org/GLM-4.6V
Updated
5 days ago
•
3.23k
•
291
Browse 1M+ models
Spaces
Running
on
Zero
532
Z Image Turbo
🖼
532
Generate stunning images from text prompts
Running
on
Zero
MCP
Featured
1.35k
Z Image Turbo
🏃
1.35k
Generate images from text prompts
Running
on
Zero
Featured
941
Dream

In [12]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [14]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [15]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 6 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nTongyi-MAI/Z-Image-Turbo\nUpdated\n5 days ago\n•\n269k\n•\n2.64k\nmicrosoft/VibeVoice-Realtime-0.5B\nUpdated\n1 day ago\n•\n120k\n•\n801\nzai-org/GLM-4.6V-Flash\nUpdated\n4 days ago\n•\n54.1k\n•\n390\nmistralai/Devstral-Small-2-24B-Instruct-2512\nUpdated\n1 day ago\n•\n13.7k\n•\n313\nzai-org/GLM-4.6V\nUpdated\n5 days ago\n•\n3.23k\n•\n291\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\n532\n

In [16]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [17]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


# Hugging Face Brochure

---

## About Hugging Face

Hugging Face is a pioneering AI company and a vibrant community focused on building the future of machine learning. It serves as the central collaboration platform where machine learning engineers, scientists, and enthusiasts can discover, share, and experiment with open-source models, datasets, and applications. Hugging Face empowers the next generation to build an open, ethical, and innovative AI ecosystem.

---

## Our Platform

- **Models**: Explore and use from over 1 million machine learning models across various modalities including text, image, video, audio, and 3D.
- **Datasets**: Access and contribute to a repository of more than 250,000 datasets tailored for a wide range of AI applications.
- **Spaces**: Host and run ML-powered applications seamlessly on the platform, with features like real-time image generation, speech synthesis, and video editing powered by AI.
- **Community Hub**: Collaborate with a fast-growing network of ML practitioners who build, improve, and share resources collectively.
- **Enterprise & Compute Solutions**: Accelerate workflows with paid compute resources and enterprise-grade tools tailored for teams and organizations.

---

## Company Culture

Hugging Face is built on the principles of:
- **Collaboration**: Open sharing and co-creation is at the heart of the platform, enabling the community to move faster and innovate together.
- **Openness and Ethics**: Committed to open source and ethical AI development for transparency and global impact.
- **Innovation**: Supporting cutting-edge research and practical applications alike by providing a powerful open-source stack.
- **Inclusivity**: Fostering a diverse and welcoming community where everyone can build and share their AI work.

---

## Our Customers & Users

Hugging Face supports a wide spectrum of users:
- **Machine Learning Engineers & Data Scientists** building next-gen AI solutions.
- **Researchers** accessing state-of-the-art models and datasets.
- **Enterprise Teams** that require managed ML infrastructure and collaboration tools.
- **Developers and End Users** who leverage AI applications from text generation to computer vision and beyond.
- **Organizations focusing on ethical and open AI practices**.

---

## Careers at Hugging Face

Join Hugging Face to be part of a visionary team shaping the upcoming AI landscape. Working with us means:
- Collaborating with world-class AI researchers, engineers, and contributors.
- Contributing to impactful open source projects used by millions worldwide.
- Being part of a culture that values transparency, creativity, and community.
- Accessing opportunities to work on challenging problems involving large-scale machine learning, natural language processing, and multi-modal AI.

*Check the Hugging Face website for the latest job openings and internship programs.*

---

## Why Choose Hugging Face?

- Access to the **largest community-driven ML repository**.
- Collaborative environment promoting **learning and portfolio-building**.
- Solutions supporting **all AI modalities**: text, vision, audio, video, and 3D.
- Enterprise-ready tools ensuring **scalability and security**.
- A **trusted brand** dedicated to an ethical AI future.

---

## Get Started

Explore millions of AI models, datasets, and applications.  
Build your ML profile and accelerate your projects with Hugging Face’s open-source platform and enterprise solutions.

**Sign up today at [huggingface.co](https://huggingface.co)!**

---

*Hugging Face – The AI community building the future.*  
Your journey into machine learning starts here.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [18]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [19]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the AI community building the future—a leading collaboration platform where machine learning enthusiasts, researchers, and enterprises come together to create, share, and innovate on models, datasets, and applications. Serving as the home of machine learning, Hugging Face empowers a global community to build an open and ethical AI future.

---

## What We Offer

- **Models:** Access and contribute to over **1 million models** spanning various modalities including text, image, video, audio, and even 3D.
- **Datasets:** Explore and share from more than **250,000 datasets** used for training and benchmarking machine learning solutions.
- **Spaces:** Deploy and host AI applications with ease through **Hugging Face Spaces**, supporting rapid prototyping and sharing of applications.
- **Community Collaboration:** Collaborate on open-source projects while building your professional ML profile. Learn from peers, contribute to trending projects, and stay at the forefront of AI innovation.
- **Enterprise Solutions:** Tailored paid compute and enterprise-grade services designed to accelerate machine learning workflows for teams and organizations.

---

## Culture and Values

- **Open and Ethical AI:** Committed to fostering an open-source ecosystem where transparency, collaboration, and ethics guide AI development.
- **Community-Driven:** Powered by one of the fastest-growing machine learning communities globally, enabling continuous learning and exchanging ideas.
- **Empowerment:** Providing tools and environments to help ML engineers, scientists, and users innovate rapidly and share their contributions.
- **Diversity of Modalities:** Supporting a wide range of AI fields from natural language processing to computer vision and beyond.

---

## Our Customers

Hugging Face serves a broad spectrum of users including:

- Independent ML Engineers and Researchers developing cutting-edge models.
- Enterprises looking for scalable, advanced AI platforms to accelerate their workflows.
- Developers and data scientists seeking to leverage community models, datasets, and applications.
- Educational institutions and students building portfolios and advancing AI knowledge.

---

## Careers

Join Hugging Face and work at the forefront of AI!

- **Collaborative Environment:** Work alongside passionate, innovative AI researchers and engineers.
- **Growth and Learning:** Engage with an active global ML community and contribute to open-source projects.
- **Impactful Work:** Help build technologies shaping the future of artificial intelligence.
- **Opportunities Include:** Roles in engineering, research, product development, community management, and enterprise solutions.

Explore open positions and apply through the Hugging Face website.

---

## Join Us

Become part of the AI revolution.  
**Sign Up** to explore models, datasets, AI apps, or start sharing your own work today at [huggingface.co](https://huggingface.co)

---

## Brand Identity

- **Colors:** Vibrant yellow (#FFD21E), orange (#FF9D00), and neutral gray (#6B7280)
- **Logo:** Recognizable and friendly, symbolizing openness and community.

---

*Hugging Face – Building the future of AI, together.*

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>